## Data Wrangling

In [1]:
# import required libraries
import random
import datetime 
import numpy as np
import pandas as pd
from random import randrange
from sklearn import preprocessing

pd.options.mode.chained_assignment = None

In [2]:
def _random_date(start,date_count):
    """This function generates a random date based on params
    Args:
        start (date object): the base date
        date_count (int): number of dates to be generated
    Returns:
        list of random dates

    """
    current = start
    while date_count > 0:
        curr = current + datetime.timedelta(days=randrange(42))
        yield curr
        date_count-=1


def generate_sample_data(row_count=100):
    """This function generates a random transaction dataset
    Args:
        row_count (int): number of rows for the dataframe
    Returns:
        a pandas dataframe

    """
    
    # sentinels
    startDate = datetime.datetime(2016, 1, 1,13)
    serial_number_sentinel = 1000
    user_id_sentinel = 5001
    product_id_sentinel = 101
    price_sentinel = 2000
    
    
    # base list of attributes
    data_dict = {
    'Serial No': np.arange(row_count)+serial_number_sentinel,
    'Date': np.random.permutation(pd.to_datetime([x.strftime("%d-%m-%Y") 
                                                    for x in _random_date(startDate,
                                                                          row_count)]).date
                                  ),
    'User ID': np.random.permutation(np.random.randint(0,
                                                       row_count,
                                                       size=int(row_count/10)) + user_id_sentinel).tolist()*10,
    'Product ID': np.random.permutation(np.random.randint(0,
                                                          row_count,
                                                          size=int(row_count/10))+ product_id_sentinel).tolist()*10 ,
    'Quantity Purchased': np.random.permutation(np.random.randint(1,
                                                                  42,
                                                                  size=row_count)),
    'Price': np.round(np.abs(np.random.randn(row_count)+1)*price_sentinel,
                      decimals=2),
    'User Type':np.random.permutation([chr(random.randrange(97, 97 + 3 + 1)) 
                                            for i in range(row_count)])
    }
    
    # introduce missing values
    for index in range(int(np.sqrt(row_count))): 
        data_dict['Price'][np.argmax(data_dict['Price'] == random.choice(data_dict['Price']))] = np.nan
        data_dict['User Type'][np.argmax(data_dict['User Type'] == random.choice(data_dict['User Type']))] = np.nan
        data_dict['Date'][np.argmax(data_dict['Date'] == random.choice(data_dict['Date']))] = np.nan
        data_dict['Product ID'][np.argmax(data_dict['Product ID'] == random.choice(data_dict['Product ID']))] = 0
        data_dict['Serial No'][np.argmax(data_dict['Serial No'] == random.choice(data_dict['Serial No']))] = -1
        data_dict['User ID'][np.argmax(data_dict['User ID'] == random.choice(data_dict['User ID']))] = -101
        
    
    # create data frame
    df = pd.DataFrame(data_dict)
    
    return df
    

def describe_dataframe(df=pd.DataFrame()):
    """This function generates descriptive stats of a dataframe
    Args:
        df (dataframe): the dataframe to be analyzed
    Returns:
        None

    """
    print("\n\n")
    print("*"*30)
    print("About the Data")
    print("*"*30)
    
    print("Number of rows::",df.shape[0])
    print("Number of columns::",df.shape[1])
    print("\n")
    
    print("Column Names::",df.columns.values.tolist())
    print("\n")
    
    print("Column Data Types::\n",df.dtypes)
    print("\n")
    
    print("Columns with Missing Values::",df.columns[df.isnull().any()].tolist())
    print("\n")
    
    print("Number of rows with Missing Values::",len(pd.isnull(df).any(1).nonzero()[0].tolist()))
    print("\n")
    
    print("Sample Indices with missing data::",pd.isnull(df).any(1).nonzero()[0].tolist()[0:5])
    print("\n")
    
    print("General Stats::")
    print(df.info())
    print("\n")
    
    print("Summary Stats::")
    print(df.describe())
    print("\n")
    
    print("Dataframe Sample Rows::")
    display(df.head(5))
    
def cleanup_column_names(df,rename_dict={},do_inplace=True):
    """This function renames columns of a pandas dataframe
       It converts column names to snake case if rename_dict is not passed. 
    Args:
        rename_dict (dict): keys represent old column names and values point to 
                            newer ones
        do_inplace (bool): flag to update existing dataframe or return a new one
    Returns:
        pandas dataframe if do_inplace is set to False, None otherwise

    """
    if not rename_dict:
        return df.rename(columns={col: col.lower().replace(' ','_') 
                    for col in df.columns.values.tolist()}, 
                  inplace=do_inplace)
    else:
        return df.rename(columns=rename_dict,inplace=do_inplace)

def expand_user_type(u_type):
    """This function maps user types to user classes
    Args:
        u_type (str): user type value
    Returns:
        (str) user_class value

    """
    if u_type in ['a','b']:
        return 'new'
    elif u_type == 'c':
        return 'existing'
    elif u_type == 'd':
        return 'loyal_existing'
    else:
        return 'error'

In [3]:
df = generate_sample_data(row_count=1000)

In [4]:
describe_dataframe(df)




******************************
About the Data
******************************
Number of rows:: 1000
Number of columns:: 7


Column Names:: ['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


Column Data Types::
 Serial No               int32
Date                   object
User ID                 int64
Product ID              int64
Quantity Purchased      int32
Price                 float64
User Type              object
dtype: object


Columns with Missing Values:: ['Date', 'Price']


Number of rows with Missing Values:: 62


Sample Indices with missing data:: [0, 2, 3, 5, 8]


General Stats::
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
Serial No             1000 non-null int32
Date                  969 non-null object
User ID               1000 non-null int64
Product ID            1000 non-null int64
Quantity Purchased    1000 non-null int32
Price                 969 non-null float64


C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
C:\Users\asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:101: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead


,Serial No,Date,User ID,Product ID,Quantity Purchased,Price,User Type
0,1000,NaN,-101,0,3,1210.88,n
1,1001,2016-01-19,5763,898,2,1441.29,n
2,1002,NaN,5358,896,18,4778.68,n
3,1003,2016-08-02,5691,442,6,NaN,n
4,1004,2016-05-02,5492,227,17,3448.13,n


### rename columns

In [5]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['Serial No', 'Date', 'User ID', 'Product ID', 'Quantity Purchased', 'Price', 'User Type']


In [7]:
#sorting in ascending order the row attributes
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

KeyError: 'serial_no'

In [8]:
cleanup_column_names(df)

In [9]:
print("Dataframe columns:\n{}".format(df.columns.tolist()))

Dataframe columns:
['serial_no', 'date', 'user_id', 'product_id', 'quantity_purchased', 'price', 'user_type']


In [10]:
#ascending order sorting for rows
display(df.sort_values(['serial_no', 'price'], 
                         ascending=[True, False]).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
367,-1,2016-01-13,5672,594,40,6010.70,b
964,-1,2016-01-17,5508,334,12,4112.63,c
176,-1,2016-01-17,5156,273,33,4100.23,b
615,-1,2016-10-02,5349,286,40,3630.10,b
317,-1,2016-01-23,5729,610,40,3355.91,a


In [11]:
#rearranging columns in data frame
display(df[['serial_no','date','user_id','user_type',
              'product_id','quantity_purchased','price']].head())

,serial_no,date,user_id,user_type,product_id,quantity_purchased,price
0,1000,NaN,-101,n,0,3,1210.88
1,1001,2016-01-19,5763,n,898,2,1441.29
2,1002,NaN,5358,n,896,18,4778.68
3,1003,2016-08-02,5691,n,442,6,NaN
4,1004,2016-05-02,5492,n,227,17,3448.13


## Filtering Columns

In [12]:
# print 10 values from column at index 3
print(df.iloc[:,3].values[0:10])

[  0 898 896 442 227 196 295 421 583 981]


In [13]:
# print 10 values of quantity  by using column name
print(df.quantity_purchased.values[0:10])

[ 3  2 18  6 17  1 19 31 35 19]


In [14]:
# print 10 values of columns with data type float using data type
print(df.select_dtypes(include=['float64']).values[:10,0])

[1210.88 1441.29 4778.68     nan 3448.13  349.46 2330.87 1753.87  960.61
 2911.45]


## filtering Rows

In [15]:
display(df.iloc[[10,501,20]])

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
10,1010,2016-07-01,5914,975,6,876.55,n
501,1501,2016-01-15,5763,898,1,4918.98,c
20,1020,NaN,5861,389,27,2831.71,n


In [16]:
# excluding specific row indices
display(df.drop([0,24,51], axis=0).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
1,1001,2016-01-19,5763,898,2,1441.29,n
2,1002,NaN,5358,896,18,4778.68,n
3,1003,2016-08-02,5691,442,6,NaN,n
4,1004,2016-05-02,5492,227,17,3448.13,n
5,1005,NaN,5811,196,1,349.46,n


In [17]:
# conditional filtering
display(df[df.quantity_purchased>25].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
7,1007,2016-01-30,5491,421,31,1753.87,n
8,1008,NaN,5294,583,35,960.61,n
11,1011,NaN,5688,818,37,221.30,n
12,1012,2016-01-30,5298,354,41,1022.79,n
13,1013,2016-09-01,5165,957,30,5454.48,n


In [18]:
# display values after row 100
display(df[100:].head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type
100,1100,2016-10-01,5861,116,1,4160.30,c
101,1101,2016-01-15,5763,898,27,858.68,d
102,1102,2016-01-21,5358,896,31,3992.89,b
103,1103,2016-01-23,5691,442,36,5948.33,d
104,1104,2016-01-26,5492,227,13,1844.06,c


## Data Type Conversion

In [19]:
df['date'] = pd.to_datetime(df.date)
# compare dtypes of the original df with this one
print(df.dtypes)

serial_no                      int32
date                  datetime64[ns]
user_id                        int64
product_id                     int64
quantity_purchased             int32
price                        float64
user_type                     object
dtype: object


In [20]:
df['user_class'] = df['user_type'].map(expand_user_type)
display(df.tail())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
995,1995,2016-11-02,5904,582,23,3297.96,b,new
996,1996,2016-01-24,5086,602,14,2566.05,a,new
997,1997,2016-01-31,5637,1021,15,973.05,b,new
998,1998,2016-01-13,5431,207,2,1118.85,a,new
999,1999,2016-01-28,5857,586,8,1431.09,c,existing


## Missing Values

In [21]:
# drop rows with missing values
df_dropped = df.dropna(subset=['date'])
display(df_dropped.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
1,1001,2016-01-19,5763,898,2,1441.29,n,error
3,1003,2016-08-02,5691,442,6,NaN,n,error
4,1004,2016-05-02,5492,227,17,3448.13,n,error
6,1006,2016-11-02,5873,295,19,2330.87,n,error
7,1007,2016-01-30,5491,421,31,1753.87,n,error


In [22]:
# fill places with mean prices where price values are missing
df_dropped['price'].fillna(value=np.round(df.price.mean(),decimals=2),
                                inplace=True)

In [23]:
# fill missing values with previous row values
df_dropped['user_type'].fillna(method='ffill',inplace=True)

In [24]:
# fill missing values with next row values
df_dropped['user_type'].fillna(method='bfill',inplace=True)

## Duplicate

In [26]:
# drop duplicate serial no rows
display(df_dropped[df_dropped.duplicated(subset=['serial_no'])].head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
95,-1,2016-01-29,5904,582,5,1823.99,d,loyal_existing
176,-1,2016-01-17,5156,273,33,4100.23,b,new
212,-1,2016-06-02,5298,354,35,2085.78,b,new
273,-1,2016-02-01,5471,350,19,460.99,d,loyal_existing
317,-1,2016-01-23,5729,610,40,3355.91,a,new


Shape of df=(969, 8)


In [27]:
df_dropped.drop_duplicates(subset=['serial_no'],inplace=True)

In [28]:
# updated dataframe
display(df_dropped.head())
print("Shape of df={}".format(df_dropped.shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
1,1001,2016-01-19,5763,898,2,1441.29,n,error
3,1003,2016-08-02,5691,442,6,2281.30,n,error
4,1004,2016-05-02,5492,227,17,3448.13,n,error
6,1006,2016-11-02,5873,295,19,2330.87,n,error
7,1007,2016-01-30,5491,421,31,1753.87,n,error


Shape of df=(941, 8)


In [29]:
# remove rows that have missing values for 3 or more missing values
display(df.dropna(thresh=3).head())
print("Shape of df={}".format(df.dropna(thresh=3).shape))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class
0,1000,NaT,-101,0,3,1210.88,n,error
1,1001,2016-01-19,5763,898,2,1441.29,n,error
2,1002,NaT,5358,896,18,4778.68,n,error
3,1003,2016-08-02,5691,442,6,NaN,n,error
4,1004,2016-05-02,5492,227,17,3448.13,n,error


Shape of df=(1000, 8)


### One hot Encoding using dummies

In [30]:
display(pd.get_dummies(df,columns=['user_type']).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_class,user_type_a,user_type_b,user_type_c,user_type_d,user_type_n
0,1000,NaT,-101,0,3,1210.88,error,0,0,0,0,1
1,1001,2016-01-19,5763,898,2,1441.29,error,0,0,0,0,1
2,1002,NaT,5358,896,18,4778.68,error,0,0,0,0,1
3,1003,2016-08-02,5691,442,6,NaN,error,0,0,0,0,1
4,1004,2016-05-02,5492,227,17,3448.13,error,0,0,0,0,1


In [31]:
type_map={'a':0,'b':1,'c':2,'d':3,np.NAN:-1}
df['encoded_user_type'] = df.user_type.map(type_map)
display((df.tail()))

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,encoded_user_type
995,1995,2016-11-02,5904,582,23,3297.96,b,new,1.0
996,1996,2016-01-24,5086,602,14,2566.05,a,new,0.0
997,1997,2016-01-31,5637,1021,15,973.05,b,new,1.0
998,1998,2016-01-13,5431,207,2,1118.85,a,new,0.0
999,1999,2016-01-28,5857,586,8,1431.09,c,existing,2.0


### Random Sampling

In [32]:
display(df.sample(frac=0.2, replace=True, random_state=42).head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,encoded_user_type
102,1102,2016-01-21,5358,896,31,3992.89,b,new,1.0
435,1435,2016-10-02,5073,561,31,916.69,a,new,0.0
860,1860,2016-01-25,5210,1036,31,2222.81,d,loyal_existing,3.0
270,1270,2016-01-21,5193,837,39,4106.23,a,new,0.0
106,1106,2016-09-01,5873,295,8,1266.83,a,new,0.0


In [33]:
df_normalized = df.dropna().copy()
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(df_normalized['price'].values.reshape(-1,1))
df_normalized['price'] = np_scaled.reshape(-1,1)

In [34]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,encoded_user_type
25,1025,2016-07-02,5947,724,38,0.121558,c,existing,2.0
28,1028,2016-01-24,5729,365,8,0.430469,c,existing,2.0
31,1031,2016-01-25,5287,423,11,0.446637,a,new,0.0
32,1032,2016-01-28,5478,587,19,0.270123,b,new,1.0
33,1033,2016-07-02,5026,693,27,0.081741,c,existing,2.0


### Data Summarization
##### Condition based aggregation

In [35]:
print("Mean price of items purchased by user_type=a :: {}".format(df['price'][df['user_type']=='a'].mean()))

Mean price of items purchased by user_type=a :: 2320.0688796680497


In [37]:

print(df['purchase_week'].value_counts())

KeyError: 'purchase_week'

In [38]:
df_normalized = df.dropna().copy()
robust_scaler = preprocessing.RobustScaler()
rs_scaled = robust_scaler.fit_transform(df_normalized['quantity_purchased'].values.reshape(-1,1))
df_normalized['quantity_purchased'] = rs_scaled.reshape(-1,1)

In [39]:
display(df_normalized.head())

,serial_no,date,user_id,product_id,quantity_purchased,price,user_type,user_class,encoded_user_type
25,1025,2016-07-02,5947,724,0.809524,1179.56,c,existing,2.0
28,1028,2016-01-24,5729,365,-0.619048,4169.34,c,existing,2.0
31,1031,2016-01-25,5287,423,-0.476190,4325.82,a,new,0.0
32,1032,2016-01-28,5478,587,-0.095238,2617.44,b,new,1.0
33,1033,2016-07-02,5026,693,0.285714,794.20,c,existing,2.0


In [40]:
print(df['purchase_week'].value_counts())

KeyError: 'purchase_week'

In [41]:
print(df.groupby(['user_class'])['quantity_purchased'].sum())

user_class
error               614
existing           5307
loyal_existing     4131
new               10935
Name: quantity_purchased, dtype: int32


In [42]:
#Group by different function
display(df.groupby(['user_class'])['quantity_purchased'].agg([np.sum,
                                                                np.mean,
                                                                np.count_nonzero]))

,sum,mean,count_nonzero
user_class,,,
error,614,20.466667,30
existing,5307,20.730469,256
loyal_existing,4131,19.394366,213
new,10935,21.826347,501


In [43]:
#Group by each attribute
display(df.groupby(['user_class','user_type']).agg({'price':np.mean,
                                                        'quantity_purchased':np.max}))

price  quantity_purchased
user_class     user_type                                 
error          n          2389.228621                  41
existing       c          2233.233387                  41
loyal_existing d          2353.936029                  41
new            a          2320.068880                  41
               b          2219.072227                  41

### Pivot Table

In [45]:
display(df.pivot_table(index='date', columns='user_type', 
                         values='price',aggfunc=np.mean))

user_type,a,b,c,d,n
date,,,,,
2016-01-01,1172.006667,736.785000,2732.185000,3242.052000,NaN
2016-01-02,2154.998000,2438.504286,3410.580000,1607.255000,NaN
2016-01-13,2063.042857,1714.803333,2358.487500,2220.764000,NaN
2016-01-14,2035.215000,2452.012500,2144.764286,1262.297500,NaN
2016-01-15,2704.510000,2739.723333,2243.435000,1780.790000,NaN
2016-01-16,2167.166667,2445.985000,175.975000,2447.473333,NaN
2016-01-17,1493.020000,2711.101667,2873.464000,2563.130000,NaN
2016-01-18,5233.170000,2173.563333,1763.718000,2115.112000,2911.4500
2016-01-19,2467.864000,2032.688000,2731.185000,155.670000,2888.4150


## Stack Dataframe

In [46]:
print(df.stack())

0    serial_no                            1000
     user_id                              -101
     product_id                              0
     quantity_purchased                      3
     price                             1210.88
     user_type                               n
     user_class                          error
1    serial_no                            1001
     date                  2016-01-19 00:00:00
     user_id                              5763
     product_id                            898
     quantity_purchased                      2
     price                             1441.29
     user_type                               n
     user_class                          error
2    serial_no                            1002
     user_id                              5358
     product_id                            896
     quantity_purchased                     18
     price                             4778.68
     user_type                               n
     user_cla